In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('results/experiment1_20211208150950.csv')

display(df)

,Unnamed: 0,write_latencies_ordering_after_write,read_latencies_ordering_after_write,write_latencies_ordering_before_write,read_latencies_ordering_before_write
0,0,0.00023,0.00008,0.00021,0.00008
1,1,0.00023,0.00008,0.00021,0.00007
2,2,0.00022,0.00008,0.00022,0.00008
3,3,0.00021,0.00008,0.00021,0.00008
4,4,0.00020,0.00007,0.00021,0.00007
5,5,0.00020,0.00008,0.00021,0.00009
6,6,0.00021,0.00008,0.00021,0.00008
7,7,0.00021,0.00008,0.00020,0.00009
8,8,0.00021,0.00008,0.00021,0.00009
9,9,0.00021,0.00008,0.00020,0.00006
